In [ ]:
import ecdsa

from bits.btypes import pubkey
from bits.btypes import pubkey_hash
from bits.btypes import bitcoin_address
from bits.utils import pubkey_hash_from_bitcoin_address
from bits.tx import outpoint
from bits.tx import txout
from bits.tx import txin
from bits.tx import tx
from bits.script import script
from bits.script import p2pkh_script_pubkey

with open("key1.pem", "rb") as key_file:
    sk1 = ecdsa.SigningKey.from_pem(key_file.read())
    x1, y1 = sk1.verifying_key.pubkey.point.x(), sk1.verifying_key.pubkey.point.y()

pubkey_1 = pubkey(x1, y1)
pubkey_hash_1 = pubkey_hash(pubkey_1)
bitcoin_addr_1 = bitcoin_address(pubkey_hash_1, network="testnet")
print("key1")
print(pubkey_1)
print(pubkey_hash_1)
print(bitcoin_addr_1)
# "https://blockstream.info/testnet/address/mu7GfxRmeYNe5PjnD1ams2RJXq2MR8tn1C"

with open("key2.pem", "rb") as key_file:
    sk2 = ecdsa.SigningKey.from_pem(key_file.read())
    x2, y2 = sk2.verifying_key.pubkey.point.x(), sk2.verifying_key.pubkey.point.y()

pubkey_2 = pubkey(x2, y2)
pubkey_hash_2 = pubkey_hash(pubkey_2)
bitcoin_addr_2 = bitcoin_address(pubkey_hash_2, network="testnet")
print("key2")
print(pubkey_2)
print(pubkey_hash_2.hex())
print(bitcoin_addr_2)
# https://blockstream.info/testnet/address/moD1fCX9Sjw8mhWt6vodLiQPVsz3bWSg4y

# get txid from block explorer
# has value of 0.0001 tBTC
#   i.e. 10,000 tsatoshis

# internal byte order (i.e. reverse byte order as shown from block explorer GUI)
txid_ = bytearray.fromhex("2f4c7fedbcc7702d47fed4b4e095e0d79d60077d66a57d310aa456a2ca27db7c")
txid_.reverse()
txid = txid_.hex()
print(txid)
outpoint_1 = outpoint(bytes.fromhex(txid), 1)

# set script_sig_1 equal to script_pubkey of outpoint_1 while signing
# scriptPubKey found from block explorer
prev_script_pubkey = bytes.fromhex("76a91495171e672584b432e2d995097222db0cc9615da288ac")
script_sig_1 = Script(prev_script_pubkey)
txin_1 = txin(outpoint_1, script_sig_1)

# 5000 satoshis to bitcoin_addr_2
script_pubkey_1 = p2pkh_script_pubkey(
    pubkey_hash_2
)
txout_1 = txout(5000, script_pubkey_1)

tx_1 = tx([txin_1], [txout_1])

# print serialized tx in hex, pre-signing
print(tx_1.hex())

In [ ]:
from bits.utils import d_hash
from bits.script.constants import SIGHASH_ALL

sig_data = d_hash(tx_1 + SIGHASH_ALL.to_bytes(4, "little"))

sig = sk1.sign_digest(
    sig_data,
    sigencode=ecdsa.util.sigencode_der  # DER format
)
print(sig.hex())
print(pubkey_1.hex())

script_sig_1 = (len(sig) + 1).to_bytes(1, "little") + sig + SIGHASH_ALL.to_bytes(1, "little") + len(pubkey_1).to_bytes(1, "little") + pubkey_1
print(script_sig_1)

txin_1_prime = txin(outpoint_1, script_sig_1)
tx_1_final = tx([txin_1_prime], [txout_1])
print(tx_1_final)